In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import uproot
import pandas as pd
import numpy as np
import math
from tqdm import tqdm




from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


print(uproot.__version__)
print(pd.__version__)

5.6.2
2.2.2


In [2]:
sig_query = 'truth_vtxInside==1 and truth_nuPdg==14 and truth_isCC==1'
sel_query  = 'numu_score>0.9 and numu_cc_flag>=0'
reco_0p_query = 'reco_Kp<35'
reco_Np_query = 'reco_Kp>=35'
true_0p_query = 'true_Kp<35'
true_Np_query = 'true_Kp>=35'

In [3]:
bdt_vars = ['numu_score','numu_cc_flag']
eval_vars = ['run','subrun','event','match_isFC']
eval_mc_vars = ['truth_vtxInside','truth_nuPdg','truth_isCC']
pf_eval_vars = ['reco_protonMomentum','reco_muonMomentum']

pf_eval_mc_vars = ['truth_startMomentum','truth_mother','truth_pdg','truth_muonMomentum']
pf_eval_reco_vars = ['reco_mother', 'reco_pdg', 'reco_nuvtxX', 'reco_nuvtxY', 'reco_nuvtxZ']
kine_vars = ['kine_pio_flag', 'kine_pio_vtx_dis', 'kine_pio_energy_1', 'kine_pio_energy_2', 'kine_pio_dis_1', 'kine_pio_dis_2', 'kine_pio_angle', 'kine_pio_mass']
containment_vars = ['reco_endXYZT', 'reco_startMomentum']
pandora_signal_vars = ['true_nu_vtx_x', 'true_nu_vtx_y', 'true_nu_vtx_z', 'ccnc', 'mc_pdg', 'mc_px', 'mc_py', 'mc_pz', 'weightSplineTimesTune']

In [4]:
f_nu_overlay_run1 = uproot.open("SURPRISE_Test_Samples_v10_04_07_05_Run4b_hyper_unified_reco2_BNB_nu_overlay_may8_reco2_hist_62280499_snapshot.root")
f_nu_overlay_run1_bdt = f_nu_overlay_run1["wcpselection"]['T_BDTvars'].arrays(bdt_vars, library = 'pd')
f_nu_overlay_run1_eval = f_nu_overlay_run1["wcpselection"]["T_eval"].arrays(eval_vars+eval_mc_vars, library = 'pd')
f_nu_overlay_run1_pfeval = f_nu_overlay_run1["wcpselection"]["T_PFeval"].arrays(pf_eval_vars+pf_eval_mc_vars ,library = 'pd')
f_nu_overlay_run1_pfeval_reco = f_nu_overlay_run1["wcpselection"]["T_PFeval"].arrays(pf_eval_reco_vars, library = 'pd')
f_nu_overlay_run1_pfeval_reco_extra = f_nu_overlay_run1["wcpselection"]["T_PFeval"].arrays(containment_vars, library = 'pd')
f_nu_overlay_run1_kinevars = f_nu_overlay_run1["wcpselection"]["T_KINEvars"].arrays(kine_vars, library = 'pd')
f_nu_overlay_run1_pot = f_nu_overlay_run1["wcpselection"]["T_pot"].arrays(["pot_tor875good","runNo",'subRunNo'], library = 'pd')
pandora_signal_vars = f_nu_overlay_run1['nuselection']['NeutrinoSelectionFilter'].arrays(pandora_signal_vars, library = 'pd')
nu_overlay_run4_super_POT = np.sum(f_nu_overlay_run1_pot["pot_tor875good"].to_numpy())/1e19
nu_overlay_run4_super_df = pd.concat([f_nu_overlay_run1_bdt, pandora_signal_vars, f_nu_overlay_run1_eval, f_nu_overlay_run1_pfeval_reco_extra, f_nu_overlay_run1_pfeval, f_nu_overlay_run1_kinevars, f_nu_overlay_run1_pfeval_reco], axis=1, sort=False)

del f_nu_overlay_run1
del f_nu_overlay_run1_bdt
del f_nu_overlay_run1_eval
del f_nu_overlay_run1_pfeval
del f_nu_overlay_run1_pfeval_reco
del f_nu_overlay_run1_kinevars
del pandora_signal_vars

nu_overlay_run4_super_df["run_num"] = [1 for i in range(nu_overlay_run4_super_df.shape[0])]

nu_overlay_run4_super_df["rse_num"] = (nu_overlay_run4_super_df["run"].to_numpy() * 100_000_000_000
                         + nu_overlay_run4_super_df["subrun"].to_numpy() * 1_000_000
                         + nu_overlay_run4_super_df["event"].to_numpy())

print(nu_overlay_run4_super_df.shape[0])
nu_overlay_run4_super_df = nu_overlay_run4_super_df.drop_duplicates(subset=['rse_num'])
print(nu_overlay_run4_super_df.shape[0])



mu3 = nu_overlay_run4_super_df["reco_muonMomentum"].to_numpy() # needed to remove [3] as made key invalid
p3 = nu_overlay_run4_super_df["reco_protonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

reco_Emuon = []
reco_Eproton = []

for i in range(len(mu3)):
    if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Emuon.append(0)
    else: 
        reco_Emuon.append(mu3[i]*1000)
    
    if(p3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Eproton.append(0)        
    else: 
        reco_Eproton.append(p3[i][3]*1000-938.27) # added [3] to account for loss of index at initialization


nu_overlay_run4_super_df["reco_Emu"] = reco_Emuon
nu_overlay_run4_super_df["reco_Kp"] = reco_Eproton

mu3 = nu_overlay_run4_super_df["truth_muonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

true_Emuon = []
true_Eproton = []

for i in range(len(mu3)):
    if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
        true_Emuon.append(0)
    else: 
        true_Emuon.append(mu3[i][3]*1000)  # added [3] to account for loss of index at initialization
    


nu_overlay_run4_super_df["true_Emu"] = true_Emuon

truth_pdgs = nu_overlay_run4_super_df["truth_pdg"].to_numpy()
truth_mothers = nu_overlay_run4_super_df["truth_mother"].to_numpy()
truth_startMomentums = nu_overlay_run4_super_df["truth_startMomentum"].to_numpy()

true_Kp = []

for i in tqdm(range(nu_overlay_run4_super_df.shape[0])):

    truth_pdg_list = truth_pdgs[i]
    truth_mother_list = truth_mothers[i]
    truth_startMomentum_list = truth_startMomentums[i]
    
    Ep = 0
    
    for j in range(len(truth_pdg_list)):
        if truth_mother_list[j] == 0 and abs(truth_pdg_list[j]) == 2212 and truth_startMomentum_list[j][3]>Ep:
            Ep = truth_startMomentum_list[j][3]
            
    if(Ep<=0): 
        true_Kp.append(0)
    else: 
        true_Kp.append(Ep*1000-938.27)    

    
nu_overlay_run4_super_df["true_Kp"] = true_Kp

93745
93745


100%|█████████████████████████████████████████████████████████████████████████| 93745/93745 [00:02<00:00, 43600.23it/s]


In [5]:
f_nu_overlay_beam_off = uproot.open("hyper-unified/SURPRISE_Test_Samples_v10_04_07_05_Run4b_hyper_unified_reco2_BNB_beam_off_may8_reco2_hist_goodruns_62280841_snapshot.root")
f_nu_overlay_beam_off_bdt = f_nu_overlay_beam_off["wcpselection"]['T_BDTvars'].arrays(bdt_vars, library = 'pd')
f_nu_overlay_beam_off_eval = f_nu_overlay_beam_off["wcpselection"]["T_eval"].arrays(eval_vars, library = 'pd')
f_nu_overlay_beam_off_pfeval = f_nu_overlay_beam_off["wcpselection"]["T_PFeval"].arrays(pf_eval_vars ,library = 'pd')
f_nu_overlay_beam_off_pfeval_reco = f_nu_overlay_beam_off["wcpselection"]["T_PFeval"].arrays(pf_eval_reco_vars+containment_vars, library = 'pd')
f_nu_overlay_beam_off_kinevars = f_nu_overlay_beam_off["wcpselection"]["T_KINEvars"].arrays(kine_vars, library = 'pd')
f_nu_overlay_beam_off_pot = f_nu_overlay_beam_off["wcpselection"]["T_pot"].arrays(["pot_tor875good","runNo",'subRunNo'], library = 'pd')
f_nu_overlay_beam_off_pand = f_nu_overlay_beam_off['nuselection']['NeutrinoSelectionFilter'].arrays( ['true_nu_vtx_x', 'true_nu_vtx_y', 'true_nu_vtx_z', 'ccnc', 'mc_pdg', 'mc_px', 'mc_py', 'mc_pz'], library = 'pd')
nu_overlay_run4_super_POT = np.sum(f_nu_overlay_beam_off_pot["pot_tor875good"].to_numpy())/1e19
nu_overlay_beam_off_super_df = pd.concat([f_nu_overlay_beam_off_bdt, f_nu_overlay_beam_off_pand, f_nu_overlay_beam_off_eval, f_nu_overlay_beam_off_pfeval, f_nu_overlay_beam_off_kinevars, f_nu_overlay_beam_off_pfeval_reco], axis=1, sort=False)

del f_nu_overlay_beam_off
del f_nu_overlay_beam_off_bdt
del f_nu_overlay_beam_off_eval
del f_nu_overlay_beam_off_pfeval
del f_nu_overlay_beam_off_pfeval_reco
del f_nu_overlay_beam_off_kinevars
del f_nu_overlay_beam_off_pand

nu_overlay_beam_off_super_df["run_num"] = [1 for i in range(nu_overlay_beam_off_super_df.shape[0])]

nu_overlay_beam_off_super_df["rse_num"] = (nu_overlay_beam_off_super_df["run"].to_numpy() * 100_000_000_000
                         + nu_overlay_beam_off_super_df["subrun"].to_numpy() * 1_000_000
                         + nu_overlay_beam_off_super_df["event"].to_numpy())

print(nu_overlay_beam_off_super_df.shape[0])
nu_overlay_beam_off_super_df = nu_overlay_beam_off_super_df.drop_duplicates(subset=['rse_num'])
print(nu_overlay_beam_off_super_df.shape[0])



mu3 = nu_overlay_beam_off_super_df["reco_muonMomentum"].to_numpy() # needed to remove [3] as made key invalid
p3 = nu_overlay_beam_off_super_df["reco_protonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

reco_Emuon = []
reco_Eproton = []

for i in range(len(mu3)):
    if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Emuon.append(0)
    else: 
        reco_Emuon.append(mu3[i]*1000)
    
    if(p3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Eproton.append(0)        
    else: 
        reco_Eproton.append(p3[i][3]*1000-938.27) # added [3] to account for loss of index at initialization


nu_overlay_beam_off_super_df["reco_Emu"] = reco_Emuon
nu_overlay_beam_off_super_df["reco_Kp"] = reco_Eproton

# mu3 = nu_overlay_beam_off_super_df["truth_muonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

# true_Emuon = []
# true_Eproton = []

# for i in range(len(mu3)):
#     if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
#         true_Emuon.append(0)
#     else: 
#         true_Emuon.append(mu3[i][3]*1000)  # added [3] to account for loss of index at initialization
    


# nu_overlay_beam_off_super_df["true_Emu"] = true_Emuon

# truth_pdgs = nu_overlay_beam_off_super_df["truth_pdg"].to_numpy()
# truth_mothers = nu_overlay_beam_off_super_df["truth_mother"].to_numpy()
# truth_startMomentums = nu_overlay_beam_off_super_df["truth_startMomentum"].to_numpy()

# true_Kp = []

# for i in tqdm(range(nu_overlay_beam_off_super_df.shape[0])):

#     truth_pdg_list = truth_pdgs[i]
#     truth_mother_list = truth_mothers[i]
#     truth_startMomentum_list = truth_startMomentums[i]
    
#     Ep = 0
    
#     for j in range(len(truth_pdg_list)):
#         if truth_mother_list[j] == 0 and abs(truth_pdg_list[j]) == 2212 and truth_startMomentum_list[j][3]>Ep:
#             Ep = truth_startMomentum_list[j][3]
            
#     if(Ep<=0): 
#         true_Kp.append(0)
#     else: 
#         true_Kp.append(Ep*1000-938.27)    

    
# nu_overlay_beam_off_super_df["true_Kp"] = true_Kp

568505
568505


In [6]:
f_nu_overlay_dirt = uproot.open("hyper-unified/SURPRISE_Test_Samples_v10_04_07_05_Run4b_hyper_unified_reco2_BNB_dirt_may8_reco2_hist_62280564_snapshot.root")
f_nu_overlay_dirt_bdt = f_nu_overlay_dirt["wcpselection"]['T_BDTvars'].arrays(bdt_vars, library = 'pd')
f_nu_overlay_dirt_eval = f_nu_overlay_dirt["wcpselection"]["T_eval"].arrays(eval_vars, library = 'pd')
f_nu_overlay_dirt_pfeval = f_nu_overlay_dirt["wcpselection"]["T_PFeval"].arrays(pf_eval_vars ,library = 'pd')
f_nu_overlay_dirt_pfeval_reco = f_nu_overlay_dirt["wcpselection"]["T_PFeval"].arrays(pf_eval_reco_vars+containment_vars, library = 'pd')
f_nu_overlay_dirt_kinevars = f_nu_overlay_dirt["wcpselection"]["T_KINEvars"].arrays(kine_vars, library = 'pd')
f_nu_overlay_dirt_pot = f_nu_overlay_dirt["wcpselection"]["T_pot"].arrays(["pot_tor875good","runNo",'subRunNo'], library = 'pd')
f_nu_overlay_dirt_pand = f_nu_overlay_dirt['nuselection']['NeutrinoSelectionFilter'].arrays( ['true_nu_vtx_x', 'true_nu_vtx_y', 'true_nu_vtx_z', 'ccnc', 'mc_pdg', 'mc_px', 'mc_py', 'mc_pz'], library = 'pd')
nu_overlay_run4_super_POT = np.sum(f_nu_overlay_dirt_pot["pot_tor875good"].to_numpy())/1e19
nu_overlay_dirt_super_df = pd.concat([f_nu_overlay_dirt_bdt, f_nu_overlay_dirt_pand, f_nu_overlay_dirt_eval, f_nu_overlay_dirt_pfeval, f_nu_overlay_dirt_kinevars, f_nu_overlay_dirt_pfeval_reco], axis=1, sort=False)

del f_nu_overlay_dirt
del f_nu_overlay_dirt_bdt
del f_nu_overlay_dirt_eval
del f_nu_overlay_dirt_pfeval
del f_nu_overlay_dirt_pfeval_reco
del f_nu_overlay_dirt_kinevars
del f_nu_overlay_dirt_pand

nu_overlay_dirt_super_df["run_num"] = [1 for i in range(nu_overlay_dirt_super_df.shape[0])]

nu_overlay_dirt_super_df["rse_num"] = (nu_overlay_dirt_super_df["run"].to_numpy() * 100_000_000_000
                         + nu_overlay_dirt_super_df["subrun"].to_numpy() * 1_000_000
                         + nu_overlay_dirt_super_df["event"].to_numpy())

print(nu_overlay_dirt_super_df.shape[0])
nu_overlay_dirt_super_df = nu_overlay_dirt_super_df.drop_duplicates(subset=['rse_num'])
print(nu_overlay_dirt_super_df.shape[0])



mu3 = nu_overlay_dirt_super_df["reco_muonMomentum"].to_numpy() # needed to remove [3] as made key invalid
p3 = nu_overlay_dirt_super_df["reco_protonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

reco_Emuon = []
reco_Eproton = []

for i in range(len(mu3)):
    if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Emuon.append(0)
    else: 
        reco_Emuon.append(mu3[i]*1000)
    
    if(p3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Eproton.append(0)        
    else: 
        reco_Eproton.append(p3[i][3]*1000-938.27) # added [3] to account for loss of index at initialization


nu_overlay_dirt_super_df["reco_Emu"] = reco_Emuon
nu_overlay_dirt_super_df["reco_Kp"] = reco_Eproton

# mu3 = nu_overlay_beam_off_super_df["truth_muonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

# true_Emuon = []
# true_Eproton = []

# for i in range(len(mu3)):
#     if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
#         true_Emuon.append(0)
#     else: 
#         true_Emuon.append(mu3[i][3]*1000)  # added [3] to account for loss of index at initialization
    


# nu_overlay_beam_off_super_df["true_Emu"] = true_Emuon

# truth_pdgs = nu_overlay_beam_off_super_df["truth_pdg"].to_numpy()
# truth_mothers = nu_overlay_beam_off_super_df["truth_mother"].to_numpy()
# truth_startMomentums = nu_overlay_beam_off_super_df["truth_startMomentum"].to_numpy()

# true_Kp = []

# for i in tqdm(range(nu_overlay_beam_off_super_df.shape[0])):

#     truth_pdg_list = truth_pdgs[i]
#     truth_mother_list = truth_mothers[i]
#     truth_startMomentum_list = truth_startMomentums[i]
    
#     Ep = 0
    
#     for j in range(len(truth_pdg_list)):
#         if truth_mother_list[j] == 0 and abs(truth_pdg_list[j]) == 2212 and truth_startMomentum_list[j][3]>Ep:
#             Ep = truth_startMomentum_list[j][3]
            
#     if(Ep<=0): 
#         true_Kp.append(0)
#     else: 
#         true_Kp.append(Ep*1000-938.27)    

    
# nu_overlay_beam_off_super_df["true_Kp"] = true_Kp

45155
45155


In [7]:
f_nu_overlay_beam_on = uproot.open("hyper-unified/SURPRISE_Test_Samples_v10_04_07_05_Run4b_hyper_unified_reco2_BNB_beam_on_may8_reco2_hist_goodruns_62280934_snapshot.root")
f_nu_overlay_beam_on_bdt = f_nu_overlay_beam_on["wcpselection"]['T_BDTvars'].arrays(bdt_vars, library = 'pd')
f_nu_overlay_beam_on_eval = f_nu_overlay_beam_on["wcpselection"]["T_eval"].arrays(eval_vars, library = 'pd')
f_nu_overlay_beam_on_pfeval = f_nu_overlay_beam_on["wcpselection"]["T_PFeval"].arrays(pf_eval_vars ,library = 'pd')
f_nu_overlay_beam_on_pfeval_reco = f_nu_overlay_beam_on["wcpselection"]["T_PFeval"].arrays(pf_eval_reco_vars+containment_vars, library = 'pd')
f_nu_overlay_beam_on_kinevars = f_nu_overlay_beam_on["wcpselection"]["T_KINEvars"].arrays(kine_vars, library = 'pd')
f_nu_overlay_beam_on_pot = f_nu_overlay_beam_on["wcpselection"]["T_pot"].arrays(["pot_tor875good","runNo",'subRunNo'], library = 'pd')
f_nu_overlay_beam_on_pand = f_nu_overlay_beam_on['nuselection']['NeutrinoSelectionFilter'].arrays( ['true_nu_vtx_x', 'true_nu_vtx_y', 'true_nu_vtx_z', 'ccnc', 'mc_pdg', 'mc_px', 'mc_py', 'mc_pz'], library = 'pd')
nu_overlay_run4_super_POT = np.sum(f_nu_overlay_beam_on_pot["pot_tor875good"].to_numpy())/1e19
nu_overlay_beam_on_super_df = pd.concat([f_nu_overlay_beam_on_bdt, f_nu_overlay_beam_on_pand, f_nu_overlay_beam_on_eval, f_nu_overlay_beam_on_pfeval, f_nu_overlay_beam_on_kinevars, f_nu_overlay_beam_on_pfeval_reco], axis=1, sort=False)

del f_nu_overlay_beam_on
del f_nu_overlay_beam_on_bdt
del f_nu_overlay_beam_on_eval
del f_nu_overlay_beam_on_pfeval
del f_nu_overlay_beam_on_pfeval_reco
del f_nu_overlay_beam_on_kinevars
del f_nu_overlay_beam_on_pand

nu_overlay_beam_on_super_df["run_num"] = [1 for i in range(nu_overlay_beam_on_super_df.shape[0])]

nu_overlay_beam_on_super_df["rse_num"] = (nu_overlay_beam_on_super_df["run"].to_numpy() * 100_000_000_000
                         + nu_overlay_beam_on_super_df["subrun"].to_numpy() * 1_000_000
                         + nu_overlay_beam_on_super_df["event"].to_numpy())

print(nu_overlay_beam_on_super_df.shape[0])
nu_overlay_beam_on_super_df = nu_overlay_beam_on_super_df.drop_duplicates(subset=['rse_num'])
print(nu_overlay_beam_on_super_df.shape[0])



mu3 = nu_overlay_beam_on_super_df["reco_muonMomentum"].to_numpy() # needed to remove [3] as made key invalid
p3 = nu_overlay_beam_on_super_df["reco_protonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

reco_Emuon = []
reco_Eproton = []

for i in range(len(mu3)):
    if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Emuon.append(0)
    else: 
        reco_Emuon.append(mu3[i]*1000)
    
    if(p3[i][3]<=0): # added [3] to account for loss of index at initialization
        reco_Eproton.append(0)        
    else: 
        reco_Eproton.append(p3[i][3]*1000-938.27) # added [3] to account for loss of index at initialization


nu_overlay_beam_on_super_df["reco_Emu"] = reco_Emuon
nu_overlay_beam_on_super_df["reco_Kp"] = reco_Eproton

# mu3 = nu_overlay_beam_off_super_df["truth_muonMomentum"].to_numpy()  # needed to remove [3] as made key invalid

# true_Emuon = []
# true_Eproton = []

# for i in range(len(mu3)):
#     if(mu3[i][3]<=0): # added [3] to account for loss of index at initialization
#         true_Emuon.append(0)
#     else: 
#         true_Emuon.append(mu3[i][3]*1000)  # added [3] to account for loss of index at initialization
    


# nu_overlay_beam_off_super_df["true_Emu"] = true_Emuon

# truth_pdgs = nu_overlay_beam_off_super_df["truth_pdg"].to_numpy()
# truth_mothers = nu_overlay_beam_off_super_df["truth_mother"].to_numpy()
# truth_startMomentums = nu_overlay_beam_off_super_df["truth_startMomentum"].to_numpy()

# true_Kp = []

# for i in tqdm(range(nu_overlay_beam_off_super_df.shape[0])):

#     truth_pdg_list = truth_pdgs[i]
#     truth_mother_list = truth_mothers[i]
#     truth_startMomentum_list = truth_startMomentums[i]
    
#     Ep = 0
    
#     for j in range(len(truth_pdg_list)):
#         if truth_mother_list[j] == 0 and abs(truth_pdg_list[j]) == 2212 and truth_startMomentum_list[j][3]>Ep:
#             Ep = truth_startMomentum_list[j][3]
            
#     if(Ep<=0): 
#         true_Kp.append(0)
#     else: 
#         true_Kp.append(Ep*1000-938.27)    

    
# nu_overlay_beam_off_super_df["true_Kp"] = true_Kp

459154
459154


In [8]:
import numpy as np
def magnitude(x, y, z):
    mag = np.sqrt(x**2 + y**2 + z**2)
    return mag

In [9]:
def is_meson(pdg_code):
    abs_pdg = np.std(pdg_code)
    if abs_pdg >= 9900000:
        return False
    thousands = (abs_pdg / 1000) % 10
    if thousands != 0:
        return False
    hundreds = (abs_pdg / 100) % 10
    if hundreds == 0:
        return False
    if abs_pdg >= 901 and abs_pdg <= 930:
        return False
    if abs_pdg == 110 or abs_pdg == 990:
        return False
    if abs_pdg == 998 or abs_pdg == 999:
        return False
    if abs_pdg == 100:
        return False
    else:
        return True

In [10]:

truth_vtxInside = nu_overlay_run4_super_df["truth_vtxInside"].to_numpy()
truth_nuPdg = nu_overlay_run4_super_df["truth_nuPdg"].to_numpy()
truth_isCC = nu_overlay_run4_super_df["truth_isCC"].to_numpy()

numu_score = nu_overlay_run4_super_df["numu_score"].to_numpy()
numu_cc_flag = nu_overlay_run4_super_df["numu_cc_flag"].to_numpy()

match_isFC = nu_overlay_run4_super_df["match_isFC"].to_numpy()

reco_Kp = nu_overlay_run4_super_df["reco_Kp"].to_numpy()
true_Kp = nu_overlay_run4_super_df["true_Kp"].to_numpy()
reco_mother = nu_overlay_run4_super_df["reco_mother"].to_numpy()
reco_pdg = nu_overlay_run4_super_df["reco_pdg"].to_numpy()

reco_endXYZT = nu_overlay_run4_super_df["reco_endXYZT"].to_numpy()
reco_startMomentum = nu_overlay_run4_super_df['reco_startMomentum'].to_numpy()
reco_muonMomentum = nu_overlay_run4_super_df['reco_muonMomentum'].to_numpy()
reco_protonMomentum = nu_overlay_run4_super_df['reco_protonMomentum'].to_numpy()

run = nu_overlay_run4_super_df["run"].to_numpy()
subrun = nu_overlay_run4_super_df["subrun"].to_numpy()
event = nu_overlay_run4_super_df["event"].to_numpy()

true_nu_vtx_x = nu_overlay_run4_super_df['true_nu_vtx_x'].to_numpy()
true_nu_vtx_y = nu_overlay_run4_super_df['true_nu_vtx_y'].to_numpy()
true_nu_vtx_z = nu_overlay_run4_super_df['true_nu_vtx_z'].to_numpy()
mc_pdg = nu_overlay_run4_super_df['mc_pdg'].to_numpy()
truth_muonMomentum = nu_overlay_run4_super_df["truth_muonMomentum"].to_numpy()

reco_nuvtxX = nu_overlay_run4_super_df["reco_nuvtxX"].to_numpy()
reco_nuvtxY = nu_overlay_run4_super_df["reco_nuvtxY"].to_numpy()
reco_nuvtxZ = nu_overlay_run4_super_df["reco_nuvtxZ"].to_numpy()
weight = nu_overlay_run4_super_df['weightSplineTimesTune'].to_numpy()

kine_pio_flag = nu_overlay_run4_super_df['kine_pio_flag'].to_numpy()
kine_pio_vtx_dis = nu_overlay_run4_super_df['kine_pio_vtx_dis'].to_numpy()
kine_pio_energy_1 = nu_overlay_run4_super_df['kine_pio_energy_1'].to_numpy()
kine_pio_energy_2 = nu_overlay_run4_super_df['kine_pio_energy_2'].to_numpy()
kine_pio_dis_1 = nu_overlay_run4_super_df['kine_pio_dis_1'].to_numpy()
kine_pio_dis_2 = nu_overlay_run4_super_df['kine_pio_dis_2'].to_numpy()
kine_pio_angle = nu_overlay_run4_super_df['kine_pio_angle'].to_numpy()
kine_pio_mass = nu_overlay_run4_super_df['kine_pio_mass'].to_numpy()

em_charge_scale = np.ones(len(nu_overlay_run4_super_df)) # set to 1 for MC 

output_file = "Dillon_list_signal_comparison.txt" 
f = open(output_file, "a")
f.write("run  subrun  event  selXp  sel0p  selNp  sel0pi#  sel0pi0  sel0pi  selNp0pi  FC  sigPan  sigXp  sig0p  sigNp  muon_idx  contained")
f.write("\n")
sig_added = 0
pand_sig_added = len(weight)
sel_added = 0
sel_and_sig = {'selXp': [], 'sel0p': [], 'selNp': [], 'FC': [], 'signal': [], 'sel0pi#': [], 'sel0pi0': [], 'sel0pi': [], 'selNp0pi': [], 'numu_score': [], 'numu_cc_flag': [], 'contained': [], 'muon_idx': [], 'pandora_sig':[], 'weight':[], 'source':[], 'Pp':[]}
for e in tqdm(range(len(truth_vtxInside))):
    
    flag_sig = 0
    if truth_isCC[e]==1 and truth_nuPdg[e]==14 and truth_vtxInside[e]==1:
        flag_sig = 1
        sig_added += 1

    flag_sel = 0
    if numu_score[e]>0.9 and numu_cc_flag[e]>=0:
        flag_sel = 1
        sel_added += 1

    flag_reco0p = 1
    flag_recoNp = 0
    if reco_Kp[e]>=35: 
        flag_reco0p = 0
        flag_recoNp = 1

        
    flag_true0p = 1
    flag_trueNp = 0
    if true_Kp[e]>=35: 
        flag_true0p = 0
        flag_trueNp = 1 

    FC = 0
    if match_isFC[e]==True:
        FC=1

    flag_0_charged_pi = 1
    contained = 0
    muon_idx = -1
    epsilon = 1e-10
    for i in range(len(reco_startMomentum[e])):
        
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 211 and reco_startMomentum[e][i][3] > 0.01+0.1396:
            flag_0_charged_pi = 0
        pi_count = 0
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 13 and reco_startMomentum[e][i][3] > 0.01+0.1057:
            pi_count +=1
            if pi_count > 1:
                flag_0_charged_pi = 0

        if np.abs(reco_startMomentum[e][i][3] - reco_muonMomentum[e][3]) < epsilon:
            muon_idx = i
            if reco_endXYZT[e][i][0] > 10.0 and reco_endXYZT[e][i][0] < 246.35 and reco_endXYZT[e][i][1] > -106.50 and reco_endXYZT[e][i][1] < 106.50 and reco_endXYZT[e][i][2] > 10.0 and reco_endXYZT[e][i][2] < 1026.8:
                contained = 1

    flag_0_pi0 = 0
    if not (kine_pio_flag[e] == 1 and kine_pio_vtx_dis[e] < 9 
            and kine_pio_energy_1[e] * em_charge_scale[e] > 40 
            and kine_pio_energy_2[e] * em_charge_scale[e] > 25 
            and kine_pio_dis_1[e] < 110 and kine_pio_dis_2[e] < 120 
            and kine_pio_angle[e] > 0 and kine_pio_angle[e] < 174 
            and kine_pio_mass[e]* em_charge_scale[e] > 22 
            and kine_pio_mass[e] * em_charge_scale[e] < 300):
        flag_0_pi0 = 1
    pandora_signal_flag = 1
    isProton = False
    leading = -99
    mesons = 0
    for i in range(len(mc_pdg[e])):
        if mc_pdg[e][i] == 2212:
            isProton = True
            mag = magnitude(truth_startMomentums[e][i][0], truth_startMomentums[e][i][1], truth_startMomentums[e][i][2])
            if mag > leading:
                leading = mag
            if is_meson(mc_pdg[e][i]):
                mesons += 1
                break
            
    
    
    if (reco_nuvtxX[e] > 234.85 or reco_nuvtxX[e] < 21.5
        or reco_nuvtxY[e] > 95.00 or reco_nuvtxY[e] < -95.00
        or reco_nuvtxZ[e] > 966.80 or reco_nuvtxZ[e] < 21.5
        or isProton == False 
        or magnitude(truth_muonMomentum[e][0], truth_muonMomentum[e][1], truth_muonMomentum[e][2]) >1.2 
        or magnitude(truth_muonMomentum[e][0], truth_muonMomentum[e][1], truth_muonMomentum[e][2]) < 0.1
        or leading > 1.0 or leading < 0.25 or mesons != 0):
        pandora_signal_flag = 0
        pand_sig_added -= 1
    
   
    
    
    
    sel_and_sig['selXp'].append(flag_sel)
    sel_and_sig['sel0p'].append(flag_sel*flag_reco0p)
    sel_and_sig['selNp'].append(flag_sel*flag_recoNp)
    sel_and_sig['FC'].append(FC)
    sel_and_sig['signal'].append(flag_sig)
    sel_and_sig['sel0pi#'].append(flag_sel*flag_0_charged_pi)
    sel_and_sig['sel0pi0'].append(flag_sel*flag_0_pi0)
    sel_and_sig['selNp0pi'].append(flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp)
    sel_and_sig['sel0pi'].append(flag_0_pi0*flag_0_charged_pi)
    sel_and_sig['contained'].append(contained)
    sel_and_sig['muon_idx'].append(muon_idx)
    sel_and_sig['pandora_sig'].append(pandora_signal_flag)
    sel_and_sig['weight'].append(weight[e])
    sel_and_sig['numu_score'].append(numu_score[e])
    sel_and_sig['numu_cc_flag'].append(em_charge_scale[e])
    sel_and_sig['source'].append(0)
    sel_and_sig['Pp'].append(magnitude(reco_protonMomentum[e][0], reco_protonMomentum[e][1], reco_protonMomentum[e][2]))
    

    string = f"{run[e]}  {subrun[e]}  {event[e]}  {flag_sel}  {flag_sel*flag_reco0p}  {flag_sel*flag_recoNp}  {flag_sel*flag_0_charged_pi}  {flag_sel*flag_0_pi0}  {flag_sel*flag_0_pi0*flag_0_charged_pi}  {flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp}  {FC}  {pandora_signal_flag}  {flag_sig}  {flag_sig*flag_true0p}  {flag_sig*flag_trueNp}  {muon_idx}  {contained}"
    f.write(string)
    f.write("\n")
    
f.write("end")    
f.close()

print(sel_added)
print(sig_added)
print(pand_sig_added)

100%|██████████████████████████████████████████████████████████████████████████| 93745/93745 [00:30<00:00, 3034.15it/s]

19690
27908
11655


In [ ]:

# truth_vtxInside = nu_overlay_beam_off_super_df["truth_vtxInside"].to_numpy()
# truth_nuPdg = nu_overlay_beam_off_super_df["truth_nuPdg"].to_numpy()
# truth_isCC = nu_overlay_beam_off_super_df["truth_isCC"].to_numpy()

numu_score = nu_overlay_beam_off_super_df["numu_score"].to_numpy()
numu_cc_flag = nu_overlay_beam_off_super_df["numu_cc_flag"].to_numpy()

match_isFC = nu_overlay_beam_off_super_df["match_isFC"].to_numpy()

reco_Kp = nu_overlay_beam_off_super_df["reco_Kp"].to_numpy()
#true_Kp = nu_overlay_beam_off_super_df["true_Kp"].to_numpy()
reco_mother = nu_overlay_beam_off_super_df["reco_mother"].to_numpy()
reco_pdg = nu_overlay_beam_off_super_df["reco_pdg"].to_numpy()

reco_endXYZT = nu_overlay_beam_off_super_df["reco_endXYZT"].to_numpy()
reco_startMomentum = nu_overlay_beam_off_super_df['reco_startMomentum'].to_numpy()
reco_muonMomentum = nu_overlay_beam_off_super_df['reco_muonMomentum'].to_numpy()
reco_protonMomentum = nu_overlay_beam_off_super_df['reco_protonMomentum'].to_numpy()

run = nu_overlay_beam_off_super_df["run"].to_numpy()
subrun = nu_overlay_beam_off_super_df["subrun"].to_numpy()
event = nu_overlay_beam_off_super_df["event"].to_numpy()

true_nu_vtx_x = nu_overlay_beam_off_super_df['true_nu_vtx_x'].to_numpy()
true_nu_vtx_y = nu_overlay_beam_off_super_df['true_nu_vtx_y'].to_numpy()
true_nu_vtx_z = nu_overlay_beam_off_super_df['true_nu_vtx_z'].to_numpy()
mc_pdg = nu_overlay_beam_off_super_df['mc_pdg'].to_numpy()
# truth_muonMomentum = nu_overlay_beam_off_super_df["truth_muonMomentum"].to_numpy()

reco_nuvtxX = nu_overlay_beam_off_super_df["reco_nuvtxX"].to_numpy()
reco_nuvtxY = nu_overlay_beam_off_super_df["reco_nuvtxY"].to_numpy()
reco_nuvtxZ = nu_overlay_beam_off_super_df["reco_nuvtxZ"].to_numpy()
# weight = nu_overlay_beam_off_super_df['weightSplineTimesTune'].to_numpy()

em_charge_scale = np.ones(len(nu_overlay_beam_off_super_df)) # set to 1 for MC 

kine_pio_vtx_dis = nu_overlay_beam_off_super_df['kine_pio_vtx_dis'].to_numpy()
kine_pio_energy_1 = nu_overlay_beam_off_super_df['kine_pio_energy_1'].to_numpy()
kine_pio_energy_2 = nu_overlay_beam_off_super_df['kine_pio_energy_2'].to_numpy()
kine_pio_dis_1 = nu_overlay_beam_off_super_df['kine_pio_dis_1'].to_numpy()
kine_pio_dis_2 = nu_overlay_beam_off_super_df['kine_pio_dis_2'].to_numpy()
kine_pio_angle = nu_overlay_beam_off_super_df['kine_pio_angle'].to_numpy()
kine_pio_mass = nu_overlay_beam_off_super_df['kine_pio_mass'].to_numpy()
kine_pio_flag = nu_overlay_beam_off_super_df['kine_pio_flag'].to_numpy()


#output_file = "Dillon_list_signal_comparison.txt" 
#f = open(output_file, "a")
#f.write("run  subrun  event  selXp  sel0p  selNp  sel0pi#  sel0pi0  sel0pi  selNp0pi  FC  sigPan  sigXp  sig0p  sigNp  muon_idx  contained")
#f.write("\n")
sel_added = 0
contained_added = 0
noPi_added = 0
for e in tqdm(range(len(numu_score))):
    
    flag_sig = 0
    # if truth_isCC[e]==1 and truth_nuPdg[e]==14 and truth_vtxInside[e]==1:
    #     flag_sig = 1


    flag_sel = 0
    if numu_score[e]>0.9 and numu_cc_flag[e]>=0:
        flag_sel = 1
        sel_added += 1

    flag_reco0p = 1
    flag_recoNp = 0
    if reco_Kp[e]>=35: 
        flag_reco0p = 0
        flag_recoNp = 1

        
    # flag_true0p = 1
    # flag_trueNp = 0
    # if true_Kp[e]>=35: 
    #     flag_true0p = 0
    #     flag_trueNp = 1 

    FC = 0
    if match_isFC[e]==True:
        FC=1

    flag_0_charged_pi = 1
    contained = 0
    muon_idx = -1
    epsilon = 1e-10
    for i in range(len(reco_startMomentum[e])):
        
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 211 and reco_startMomentum[e][i][3] > 0.01+0.1396:
            flag_0_charged_pi = 0
        pi_count = 0
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 13 and reco_startMomentum[e][i][3] > 0.01+0.1057:
            pi_count +=1
            if pi_count > 1:
                flag_0_charged_pi = 0

        if np.abs(reco_startMomentum[e][i][3] - reco_muonMomentum[e][3]) < epsilon:
            muon_idx = i
            if reco_endXYZT[e][i][0] > 10.0 and reco_endXYZT[e][i][0] < 246.35 and reco_endXYZT[e][i][1] > -106.50 and reco_endXYZT[e][i][1] < 106.50 and reco_endXYZT[e][i][2] > 10.0 and reco_endXYZT[e][i][2] < 1026.8:
                contained = 1
                contained_added += 1

    flag_0_pi0 = 0
    if not (kine_pio_flag[e] == 1 and kine_pio_vtx_dis[e] < 9 
            and kine_pio_energy_1[e] * em_charge_scale[e] > 40 
            and kine_pio_energy_2[e] * em_charge_scale[e] > 25 
            and kine_pio_dis_1[e] < 110 and kine_pio_dis_2[e] < 120 
            and kine_pio_angle[e] > 0 and kine_pio_angle[e] < 174 
            and kine_pio_mass[e]* em_charge_scale[e] > 22 
            and kine_pio_mass[e] * em_charge_scale[e] < 300):
        flag_0_pi0 = 1
        
    pandora_signal_flag = 0
    # isProton = False
    # leading = -99
    # mesons = 0
    # for i in range(len(reco_startMomentum[e])):
    #     if mc_pdg[e][i] == 2212:
    #         isProton = True
    #         mag = magnitude(reco_startMomentum[e][i][0], reco_startMomentums[e][i][1], reco_startMomentums[e][i][2])
    #         if mag > leading:
    #             leading = mag
    #         if is_meson(mc_pdg[e][i]):
    #             mesons += 1
    #             break
            
    
    
    if (reco_nuvtxX[e] > 234.85 or reco_nuvtxX[e] < 21.5
        or reco_nuvtxY[e] > 95.00 or reco_nuvtxY[e] < -95.00
        or reco_nuvtxZ[e] > 966.80 or reco_nuvtxZ[e] < 21.5
        or isProton == False 
        or magnitude(reco_muonMomentum[e][0], reco_muonMomentum[e][1], reco_muonMomentum[e][2]) >1.2 
        or magnitude(reco_muonMomentum[e][0], reco_muonMomentum[e][1], reco_muonMomentum[e][2]) < 0.1
        or leading > 1.0 or leading < 0.25 or mesons != 0):
        pandora_signal_flag = 0
    
   
    
    
    
    sel_and_sig['selXp'].append(flag_sel)
    sel_and_sig['sel0p'].append(flag_sel*flag_reco0p)
    sel_and_sig['selNp'].append(flag_sel*flag_recoNp)
    sel_and_sig['FC'].append(FC)
    sel_and_sig['signal'].append(flag_sig)
    sel_and_sig['sel0pi#'].append(flag_sel*flag_0_charged_pi)
    sel_and_sig['sel0pi0'].append(flag_sel*flag_0_pi0)
    sel_and_sig['selNp0pi'].append(flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp)
    sel_and_sig['sel0pi'].append(flag_0_pi0*flag_0_charged_pi)
    sel_and_sig['contained'].append(contained)
    sel_and_sig['muon_idx'].append(muon_idx)
    sel_and_sig['pandora_sig'].append(pandora_signal_flag)
    sel_and_sig['weight'].append(1)
    sel_and_sig['numu_score'].append(numu_score[e])
    sel_and_sig['numu_cc_flag'].append(1)
    sel_and_sig['source'].append(1)
    sel_and_sig['Pp'].append(magnitude(reco_protonMomentum[e][0], reco_protonMomentum[e][1], reco_protonMomentum[e][2]))


    #string = f"{run[e]}  {subrun[e]}  {event[e]}  {flag_sel}  {flag_sel*flag_reco0p}  {flag_sel*flag_recoNp}  {flag_sel*flag_0_charged_pi}  {flag_sel*flag_0_pi0}  {flag_sel*flag_0_pi0*flag_0_charged_pi}  {flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp}  {FC}  {pandora_signal_flag}  {flag_sig}  {flag_sig*flag_true0p}  {flag_sig*flag_trueNp}  {muon_idx}  {contained}"
    #f.write(string)
    #f.write("\n")
    
#f.write("end")    
#f.close()
print(sel_added)
print(contained_added)
print(noPi_added)

 79%|████████████████████████████████████████████████████████▎              | 450486/568505 [00:19<00:05, 23207.51it/s]

In [ ]:

# truth_vtxInside = nu_overlay_beam_off_super_df["truth_vtxInside"].to_numpy()
# truth_nuPdg = nu_overlay_beam_off_super_df["truth_nuPdg"].to_numpy()
# truth_isCC = nu_overlay_beam_off_super_df["truth_isCC"].to_numpy()

numu_score = nu_overlay_dirt_super_df["numu_score"].to_numpy()
numu_cc_flag = nu_overlay_dirt_super_df["numu_cc_flag"].to_numpy()

match_isFC = nu_overlay_dirt_super_df["match_isFC"].to_numpy()

reco_Kp = nu_overlay_dirt_super_df["reco_Kp"].to_numpy()
#true_Kp = nu_overlay_dirt_super_df["true_Kp"].to_numpy()
reco_mother = nu_overlay_dirt_super_df["reco_mother"].to_numpy()
reco_pdg = nu_overlay_dirt_super_df["reco_pdg"].to_numpy()

reco_endXYZT = nu_overlay_dirt_super_df["reco_endXYZT"].to_numpy()
reco_startMomentum = nu_overlay_dirt_super_df['reco_startMomentum'].to_numpy()
reco_muonMomentum = nu_overlay_dirt_super_df['reco_muonMomentum'].to_numpy()
reco_protonMomentum = nu_overlay_dirt_super_df['reco_protonMomentum'].to_numpy()

run = nu_overlay_dirt_super_df["run"].to_numpy()
subrun = nu_overlay_dirt_super_df["subrun"].to_numpy()
event = nu_overlay_dirt_super_df["event"].to_numpy()

true_nu_vtx_x = nu_overlay_dirt_super_df['true_nu_vtx_x'].to_numpy()
true_nu_vtx_y = nu_overlay_dirt_super_df['true_nu_vtx_y'].to_numpy()
true_nu_vtx_z = nu_overlay_dirt_super_df['true_nu_vtx_z'].to_numpy()
mc_pdg = nu_overlay_dirt_super_df['mc_pdg'].to_numpy()
# truth_muonMomentum = nu_overlay_dirt_super_df["truth_muonMomentum"].to_numpy()

reco_nuvtxX = nu_overlay_dirt_super_df["reco_nuvtxX"].to_numpy()
reco_nuvtxY = nu_overlay_dirt_super_df["reco_nuvtxY"].to_numpy()
reco_nuvtxZ = nu_overlay_dirt_super_df["reco_nuvtxZ"].to_numpy()
# weight = nu_overlay_dirt_super_df['weightSplineTimesTune'].to_numpy()


kine_pio_vtx_dis = nu_overlay_dirt_super_df['kine_pio_vtx_dis'].to_numpy()
kine_pio_energy_1 = nu_overlay_dirt_super_df['kine_pio_energy_1'].to_numpy()
kine_pio_energy_2 = nu_overlay_dirt_super_df['kine_pio_energy_2'].to_numpy()
kine_pio_dis_1 = nu_overlay_dirt_super_df['kine_pio_dis_1'].to_numpy()
kine_pio_dis_2 = nu_overlay_dirt_super_df['kine_pio_dis_2'].to_numpy()
kine_pio_angle = nu_overlay_dirt_super_df['kine_pio_angle'].to_numpy()
kine_pio_mass = nu_overlay_dirt_super_df['kine_pio_mass'].to_numpy()
kine_pio_flag = nu_overlay_dirt_super_df['kine_pio_flag'].to_numpy()
em_charge_scale = np.ones(len(nu_overlay_dirt_super_df)) # set to 1 for MC 


#output_file = "Dillon_list_signal_comparison.txt" 
#f = open(output_file, "a")
#f.write("run  subrun  event  selXp  sel0p  selNp  sel0pi#  sel0pi0  sel0pi  selNp0pi  FC  sigPan  sigXp  sig0p  sigNp  muon_idx  contained")
#f.write("\n")

for e in tqdm(range(len(numu_score))):
    
    flag_sig = 0
    # if truth_isCC[e]==1 and truth_nuPdg[e]==14 and truth_vtxInside[e]==1:
    #     flag_sig = 1


    flag_sel = 0
    if numu_score[e]>0.9 and numu_cc_flag[e]>=0:
        flag_sel = 1

    flag_reco0p = 1
    flag_recoNp = 0
    if reco_Kp[e]>=35: 
        flag_reco0p = 0
        flag_recoNp = 1

        
    # flag_true0p = 1
    # flag_trueNp = 0
    # if true_Kp[e]>=35: 
    #     flag_true0p = 0
    #     flag_trueNp = 1 

    FC = 0
    if match_isFC[e]==True:
        FC=1

    flag_0_charged_pi = 1
    contained = 0
    muon_idx = -1
    epsilon = 1e-10
    for i in range(len(reco_startMomentum[e])):
        
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 211 and reco_startMomentum[e][i][3] > 0.01+0.1396:
            flag_0_charged_pi = 0
        pi_count = 0
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 13 and reco_startMomentum[e][i][3] > 0.01+0.1057:
            pi_count +=1
            if pi_count > 1:
                flag_0_charged_pi = 0

        if np.abs(reco_startMomentum[e][i][3] - reco_muonMomentum[e][3]) < epsilon:
            muon_idx = i
            if reco_endXYZT[e][i][0] > 10.0 and reco_endXYZT[e][i][0] < 246.35 and reco_endXYZT[e][i][1] > -106.50 and reco_endXYZT[e][i][1] < 106.50 and reco_endXYZT[e][i][2] > 10.0 and reco_endXYZT[e][i][2] < 1026.8:
                contained = 1

    flag_0_pi0 = 0
    if not (kine_pio_flag[e] == 1 and kine_pio_vtx_dis[e] < 9 
            and kine_pio_energy_1[e] * em_charge_scale[e] > 40 
            and kine_pio_energy_2[e] * em_charge_scale[e] > 25 
            and kine_pio_dis_1[e] < 110 and kine_pio_dis_2[e] < 120 
            and kine_pio_angle[e] > 0 and kine_pio_angle[e] < 174 
            and kine_pio_mass[e]* em_charge_scale[e] > 22 
            and kine_pio_mass[e] * em_charge_scale[e] < 300):
        flag_0_pi0 = 1
    pandora_signal_flag = 0
    # isProton = False
    # leading = -99
    # mesons = 0
    # for i in range(len(mc_pdg[e])):
    #     if mc_pdg[e][i] == 2212:
    #         isProton = True
    #         mag = magnitude(reco_startMomentum[e][i][0], reco_startMomentum[e][i][1], reco_startMomentum[e][i][2])
    #         if mag > leading:
    #             leading = mag
    #         if is_meson(mc_pdg[e][i]):
    #             mesons += 1
    #             break
            
    
    
    if (reco_nuvtxX[e] > 234.85 or reco_nuvtxX[e] < 21.5
        or reco_nuvtxY[e] > 95.00 or reco_nuvtxY[e] < -95.00
        or reco_nuvtxZ[e] > 966.80 or reco_nuvtxZ[e] < 21.5
        or isProton == False 
        or magnitude(reco_muonMomentum[e][0], reco_muonMomentum[e][1], reco_muonMomentum[e][2]) >1.2 
        or magnitude(reco_muonMomentum[e][0], reco_muonMomentum[e][1], reco_muonMomentum[e][2]) < 0.1
        or leading > 1.0 or leading < 0.25 or mesons != 0):
        pandora_signal_flag = 0

   
    
    
    
    sel_and_sig['selXp'].append(flag_sel)
    sel_and_sig['sel0p'].append(flag_sel*flag_reco0p)
    sel_and_sig['selNp'].append(flag_sel*flag_recoNp)
    sel_and_sig['FC'].append(FC)
    sel_and_sig['signal'].append(flag_sig)
    sel_and_sig['sel0pi#'].append(flag_sel*flag_0_charged_pi)
    sel_and_sig['sel0pi0'].append(flag_sel*flag_0_pi0)
    sel_and_sig['selNp0pi'].append(flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp)
    sel_and_sig['sel0pi'].append(flag_0_pi0*flag_0_charged_pi)
    sel_and_sig['contained'].append(contained)
    sel_and_sig['muon_idx'].append(muon_idx)
    sel_and_sig['pandora_sig'].append(pandora_signal_flag)
    sel_and_sig['weight'].append(1)
    sel_and_sig['numu_score'].append(numu_score[e])
    sel_and_sig['numu_cc_flag'].append(1)
    sel_and_sig['source'].append(2)
    sel_and_sig['Pp'].append(magnitude(reco_protonMomentum[e][0], reco_protonMomentum[e][1], reco_protonMomentum[e][2]))


    #string = f"{run[e]}  {subrun[e]}  {event[e]}  {flag_sel}  {flag_sel*flag_reco0p}  {flag_sel*flag_recoNp}  {flag_sel*flag_0_charged_pi}  {flag_sel*flag_0_pi0}  {flag_sel*flag_0_pi0*flag_0_charged_pi}  {flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp}  {FC}  {pandora_signal_flag}  {flag_sig}  {flag_sig*flag_true0p}  {flag_sig*flag_trueNp}  {muon_idx}  {contained}"
    #f.write(string)
    #f.write("\n")
    
#f.write("end")    
#f.close()


In [ ]:

# truth_vtxInside = nu_overlay_beam_off_super_df["truth_vtxInside"].to_numpy()
# truth_nuPdg = nu_overlay_beam_off_super_df["truth_nuPdg"].to_numpy()
# truth_isCC = nu_overlay_beam_off_super_df["truth_isCC"].to_numpy()

numu_score = nu_overlay_beam_on_super_df["numu_score"].to_numpy()
numu_cc_flag = nu_overlay_beam_on_super_df["numu_cc_flag"].to_numpy()

match_isFC = nu_overlay_beam_on_super_df["match_isFC"].to_numpy()

reco_Kp = nu_overlay_beam_on_super_df["reco_Kp"].to_numpy()
#true_Kp = nu_overlay_beam_on_super_df["true_Kp"].to_numpy()
reco_mother = nu_overlay_beam_on_super_df["reco_mother"].to_numpy()
reco_pdg = nu_overlay_beam_on_super_df["reco_pdg"].to_numpy()

reco_endXYZT = nu_overlay_beam_on_super_df["reco_endXYZT"].to_numpy()
reco_startMomentum = nu_overlay_beam_on_super_df['reco_startMomentum'].to_numpy()
reco_muonMomentum = nu_overlay_beam_on_super_df['reco_muonMomentum'].to_numpy()
reco_protonMomentum = nu_overlay_beam_on_super_df['reco_protonMomentum'].to_numpy()

run = nu_overlay_beam_on_super_df["run"].to_numpy()
subrun = nu_overlay_beam_on_super_df["subrun"].to_numpy()
event = nu_overlay_beam_on_super_df["event"].to_numpy()

true_nu_vtx_x = nu_overlay_beam_on_super_df['true_nu_vtx_x'].to_numpy()
true_nu_vtx_y = nu_overlay_beam_on_super_df['true_nu_vtx_y'].to_numpy()
true_nu_vtx_z = nu_overlay_beam_on_super_df['true_nu_vtx_z'].to_numpy()
mc_pdg = nu_overlay_beam_on_super_df['mc_pdg'].to_numpy()
# truth_muonMomentum = nu_overlay_beam_on_super_df["truth_muonMomentum"].to_numpy()

reco_nuvtxX = nu_overlay_beam_on_super_df["reco_nuvtxX"].to_numpy()
reco_nuvtxY = nu_overlay_beam_on_super_df["reco_nuvtxY"].to_numpy()
reco_nuvtxZ = nu_overlay_beam_on_super_df["reco_nuvtxZ"].to_numpy()
# weight = nu_overlay_beam_on_super_df['weightSplineTimesTune'].to_numpy()


kine_pio_vtx_dis = nu_overlay_beam_on_super_df['kine_pio_vtx_dis'].to_numpy()
kine_pio_energy_1 = nu_overlay_beam_on_super_df['kine_pio_energy_1'].to_numpy()
kine_pio_energy_2 = nu_overlay_beam_on_super_df['kine_pio_energy_2'].to_numpy()
kine_pio_dis_1 = nu_overlay_beam_on_super_df['kine_pio_dis_1'].to_numpy()
kine_pio_dis_2 = nu_overlay_beam_on_super_df['kine_pio_dis_2'].to_numpy()
kine_pio_angle = nu_overlay_beam_on_super_df['kine_pio_angle'].to_numpy()
kine_pio_mass = nu_overlay_beam_on_super_df['kine_pio_mass'].to_numpy()
kine_pio_flag = nu_overlay_beam_on_super_df['kine_pio_flag'].to_numpy()
em_charge_scale = np.ones(len(nu_overlay_beam_on_super_df)) # set to 1 for MC 


#output_file = "Dillon_list_signal_comparison.txt" 
#f = open(output_file, "a")
#f.write("run  subrun  event  selXp  sel0p  selNp  sel0pi#  sel0pi0  sel0pi  selNp0pi  FC  sigPan  sigXp  sig0p  sigNp  muon_idx  contained")
#f.write("\n")

for e in tqdm(range(len(numu_score))):
    
    flag_sig = 0
    # if truth_isCC[e]==1 and truth_nuPdg[e]==14 and truth_vtxInside[e]==1:
    #     flag_sig = 1


    flag_sel = 0
    if numu_score[e]>0.9 and numu_cc_flag[e]>=0:
        flag_sel = 1

    flag_reco0p = 1
    flag_recoNp = 0
    if reco_Kp[e]>=35: 
        flag_reco0p = 0
        flag_recoNp = 1

        
    # flag_true0p = 1
    # flag_trueNp = 0
    # if true_Kp[e]>=35: 
    #     flag_true0p = 0
    #     flag_trueNp = 1 

    FC = 0
    if match_isFC[e]==True:
        FC=1

    flag_0_charged_pi = 1
    contained = 0
    muon_idx = -1
    epsilon = 1e-10
    for i in range(len(reco_startMomentum[e])):
        
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 211 and reco_startMomentum[e][i][3] > 0.01+0.1396:
            flag_0_charged_pi = 0
        pi_count = 0
        if reco_mother[e][i] == 0 and reco_pdg[e][i] == 13 and reco_startMomentum[e][i][3] > 0.01+0.1057:
            pi_count +=1
            if pi_count > 1:
                flag_0_charged_pi = 0

        if np.abs(reco_startMomentum[e][i][3] - reco_muonMomentum[e][3]) < epsilon:
            muon_idx = i
            if reco_endXYZT[e][i][0] > 10.0 and reco_endXYZT[e][i][0] < 246.35 and reco_endXYZT[e][i][1] > -106.50 and reco_endXYZT[e][i][1] < 106.50 and reco_endXYZT[e][i][2] > 10.0 and reco_endXYZT[e][i][2] < 1026.8:
                contained = 1

    flag_0_pi0 = 0
    if not (kine_pio_flag[e] == 1 and kine_pio_vtx_dis[e] < 9 
            and kine_pio_energy_1[e] * em_charge_scale[e] > 40 
            and kine_pio_energy_2[e] * em_charge_scale[e] > 25 
            and kine_pio_dis_1[e] < 110 and kine_pio_dis_2[e] < 120 
            and kine_pio_angle[e] > 0 and kine_pio_angle[e] < 174 
            and kine_pio_mass[e]* em_charge_scale[e] > 22 
            and kine_pio_mass[e] * em_charge_scale[e] < 300):
        flag_0_pi0 = 1
    pandora_signal_flag = 0
    # isProton = False
    # leading = -99
    # mesons = 0
    # for i in range(len(mc_pdg[e])):
    #     if mc_pdg[e][i] == 2212:
    #         isProton = True
    #         mag = magnitude(reco_startMomentum[e][i][0], reco_startMomentum[e][i][1], reco_startMomentum[e][i][2])
    #         if mag > leading:
    #             leading = mag
    #         if is_meson(mc_pdg[e][i]):
    #             mesons += 1
    #             break
            
    
    
    if (reco_nuvtxX[e] > 234.85 or reco_nuvtxX[e] < 21.5
        or reco_nuvtxY[e] > 95.00 or reco_nuvtxY[e] < -95.00
        or reco_nuvtxZ[e] > 966.80 or reco_nuvtxZ[e] < 21.5
        or isProton == False 
        or magnitude(reco_muonMomentum[e][0], reco_muonMomentum[e][1], reco_muonMomentum[e][2]) >1.2 
        or magnitude(reco_muonMomentum[e][0], reco_muonMomentum[e][1], reco_muonMomentum[e][2]) < 0.1
        or leading > 1.0 or leading < 0.25 or mesons != 0):
        pandora_signal_flag = 0
    
   
    
    
    
    sel_and_sig['selXp'].append(flag_sel)
    sel_and_sig['sel0p'].append(flag_sel*flag_reco0p)
    sel_and_sig['selNp'].append(flag_sel*flag_recoNp)
    sel_and_sig['FC'].append(FC)
    sel_and_sig['signal'].append(flag_sig)
    sel_and_sig['sel0pi#'].append(flag_sel*flag_0_charged_pi)
    sel_and_sig['sel0pi0'].append(flag_sel*flag_0_pi0)
    sel_and_sig['selNp0pi'].append(flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp)
    sel_and_sig['sel0pi'].append(flag_0_pi0*flag_0_charged_pi)
    sel_and_sig['contained'].append(contained)
    sel_and_sig['muon_idx'].append(muon_idx)
    sel_and_sig['pandora_sig'].append(pandora_signal_flag)
    sel_and_sig['weight'].append(1)
    sel_and_sig['numu_score'].append(numu_score[e])
    sel_and_sig['numu_cc_flag'].append(1)
    sel_and_sig['source'].append(3)
    sel_and_sig['Pp'].append(magnitude(reco_protonMomentum[e][0], reco_protonMomentum[e][1], reco_protonMomentum[e][2]))


    #string = f"{run[e]}  {subrun[e]}  {event[e]}  {flag_sel}  {flag_sel*flag_reco0p}  {flag_sel*flag_recoNp}  {flag_sel*flag_0_charged_pi}  {flag_sel*flag_0_pi0}  {flag_sel*flag_0_pi0*flag_0_charged_pi}  {flag_sel*flag_0_pi0*flag_0_charged_pi*flag_recoNp}  {FC}  {pandora_signal_flag}  {flag_sig}  {flag_sig*flag_true0p}  {flag_sig*flag_trueNp}  {muon_idx}  {contained}"
    #f.write(string)
    #f.write("\n")
    
#f.write("end")    
#f.close()


In [ ]:
df_eff = pd.DataFrame(sel_and_sig)

In [ ]:
Pp_mask = df_eff['Pp'] != magnitude(-1, -1, -1)
df_eff = df_eff[Pp_mask]

In [ ]:
MC_mask = df_eff['source'] == 0
beam_off_mask = df_eff['source'] == 1
dirt_mask = df_eff['source'] == 2
beam_on_mask = df_eff['source'] == 3

MC_data = df_eff[MC_mask]
beam_off_data = df_eff[beam_off_mask]
dirt_data = df_eff[dirt_mask]
beam_on_data = df_eff[beam_on_mask]

In [ ]:
MC_selected_mask = (MC_data['selNp0pi'] == 1)
beam_off_selected_mask = beam_off_data['selNp0pi'] == 1
dirt_selected_mask = dirt_data['selNp0pi'] == 1
beam_on_selected_mask = beam_on_data['selNp0pi'] == 1

MC_selected = MC_data[MC_selected_mask]
beam_off_selected = beam_off_data[beam_off_selected_mask]
dirt_selected = dirt_data[dirt_selected_mask]
beam_on_selected = beam_on_data[beam_on_selected_mask]

In [ ]:
with uproot.recreate('wc_MC.root') as f:
    f["MCdata"] = MC_selected

with uproot.recreate('wc_beam_off.root') as f:
    f["BeamOffdata"] = beam_off_selected

with uproot.recreate('wc_dirt.root') as f:
    f["Dirtdata"] = dirt_selected

with uproot.recreate('wc_beam_on.root') as f:
    f["BeamOndata"] = beam_on_selected